In [1]:
from picamera.array import PiYUVArray, PiRGBArray
from picamera import PiCamera
from scipy.signal import find_peaks, butter, filtfilt
import time
import matplotlib.pyplot as plt
import skimage as ski
import numpy as np
import time
from pwm import PWM
from ipywidgets import interact
import PID

In [2]:
#global variables
line = 200
Kp = 1562.5
Kd = 0
Ki = 0

#setup servo and motor contol
pwm0 = PWM(0)
pwm0.period = 20000000
pwm0.duty_cycle = 11000000
pwm0.enable = True
pwm0.duty_cycle = 11000000

pwm1 = PWM(1)
pwm1.period = 20000000
pwm1.duty_cycle = 1500000
pwm1.enable = True

In [3]:
#Setup PID
#Setup PID

pid = PID.PID(Kp,Kd,Ki) #Kp is set to 2000, I & D are disabled
pid.SetPoint = 320
pid.setSampleTime(0.01) #sample time is 10ms

# start motor at low speed
pwm0.duty_cycle = 1100000

In [4]:
#define interact function
#Function that will be called for each slider movement
def changeMotorSpeed(inMotorSpeed):
    pwm0.duty_cycle = inMotorSpeed

def changeSteeringDirection(inDirection):
    pwm1.duty_cycle = inDirection

def motorOn(on):
    if not on:
        pwm0.duty_cycle = 1000000
        
def changeLine(inLine):
    line = inLine
    
 def changeKP(inKP):
    pid.setKp(inKP)
    
#  def changeKD(inKD):
#     pid.setKd(inKD)
    
def changeKI(inKI):
    pid.setKi(inKI)
    

# Create a slider and set its default position to 1000000
interact(changeMotorSpeed, inMotorSpeed=(1000000,1500000,10000))
interact(changeSteeringDirection, inDirection=(1000000,2000000,10000))
interact(changeKP, inKP=(1062.5,2062.5,50))
interact(changeKI, inKI=(-1000,1000,20))
# interact(changeKD, inKD=(0,50,2))
interact(motorOn, on=True)
interact(changeLine, inLine=(0,480,20))

# set default value
line = 200
pwm0.duty_cycle = 1000000

interactive(children=(IntSlider(value=1250000, description='inMotorSpeed', max=1500000, min=1000000, step=1000…

interactive(children=(IntSlider(value=1500000, description='inDirection', max=2000000, min=1000000, step=10000…

interactive(children=(Checkbox(value=True, description='on'), Output()), _dom_classes=('widget-interact',))

interactive(children=(IntSlider(value=240, description='inLine', max=480, step=20), Output()), _dom_classes=('…

In [5]:
# setup camera settings
res = (640, 480)
camera = PiCamera()
camera.sensor_mode = 7
camera.resolution = res
camera.framerate = 120
#initialize camera feed
rawCapture = PiYUVArray(camera, size=res)
stream = camera.capture_continuous(rawCapture, format="yuv", use_video_port=True)

In [ ]:
#test loop
pwm0.duty_cycle = 1100000
first_run = True
threshold = 100
p_old = []
p_current = []
threshold_str8 = 75
for f in stream:
    # Get the intensity component of the image (a trick to get black and white images)
    I = f.array[:, :, 0]
    # Reset the buffer for the next image
    rawCapture.truncate(0)
    # Select a horizontal line in the middle of the image
    L = I[150, :]
    # To filter the noise in the image we use a 3rd order Butterworth filter
    b, a = butter(3, 0.02) 
    # Smooth the transitions so we can detect the peaks 
    Lf = filtfilt(b, a, L)
    # Due to the noise in the image the algorithm finds many peaks heigher than 0.5
    
    p_current = find_peaks(Lf, height=100)

    if first_run:
        # calculations
        # print(p)
        pid.update(p_current[0][0])
        feedback = int(pid.output + 1500000)
        pwm1.duty_cycle = feedback
        #print(feedback)
        first_run = False
        p_old = p_current
    else:
        # good data
        if((len(p_current[0]) > 0) and (abs(p_current[0][0] - p_old[0][0]) < threshold)):
            # calculations
            # print(p)
            #print(feedback)
            print("new")
            flag = True
            # since the data is good we will look at another row further ahead
            L = I[250, :]
            Lf = filtfilt(b, a, L)
            p_250 = find_peaks(Lf, height=100)
            if(len(p_250) == 0):
                flag = False
            
            

            # since the data is good we will look at another row closer to us
            L = I[120, :]
            Lf = filtfilt(b, a, L)
            p_120 = find_peaks(Lf, height=100)
            if(len(p_120) == 0):
                flag = False

            ### logic -- what do we do?? 
            ## control the speed
            
            if (flag and (abs(p_120[0][0] - 320) < threshold_str8)) and (abs(p_250[0][0] - 320) < threshold_str8):
                pwm0.duty_cycle = 1200000
                pid.update(p_current[0][0])
                feedback = int(pid.output + 1500000)
                pwm1.duty_cycle = feedback
                p_old = p_current
            else:
                # we are not in a straight away aka "a turn", process with the data nearby
                pwm0.duty_cycle = 1100000
                pid.update(p_120[0][0])
                feedback = int(pid.output + 1500000)
                pwm1.duty_cycle = feedback
        else:
            #ignore the new image -- it is not the center line or there are no peaks
            pid.update(p_old[0][0])
            feedback = int(pid.output + 1500000)
            pwm1.duty_cycle = feedback
            #print(feedback)
            print("old")
    if(pwm0.duty_cycle < 1050000):
        plt.imshow(I)
        break

new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old
old


In [7]:
pwm0.duty_cycle = 1000000
pwm0.enable = False
pwm1.duty_cycle = 1500000
pwm1.enable = False

